# Import and Export by India from 2014 to 2017

To better understand the imports and exports by India. Some of questions I would like to be answered are

- Top countries by growth percentage.
- Top commodity by quantity or value.
- YoY growth of export and import.

The whole dataset is about import and export data available by principle commodity and country wise for 3 years from Apr'2014 to Mar'2017.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

## loading data

In [60]:
data_ep_2014 = pd.read_csv("dataset/PC_Export_2014_2015.csv")
data_ip_2014 = pd.read_csv("dataset/PC_Import_2014_2015.csv")
data_ep_2015 = pd.read_csv("dataset/PC_Export_2015_2016.csv")
data_ip_2015 = pd.read_csv("dataset/PC_Import_2015_2016.csv")
data_ep_2016 = pd.read_csv("dataset/PC_Export_2016_2017.csv")
data_ip_2016 = pd.read_csv("dataset/PC_Import_2016_2017.csv")

print data_ep_2014.head()
print data_ip_2014.head()

  pc_code     pc_description unit  country_code  country_name  quantity  \
0      99  Other Commodities   Na           599   Unspecified       NaN   
1      99  Other Commodities   Na          1213         Kenya       NaN   
2      99  Other Commodities   Na          1395  Tanzania Rep       NaN   
3      99  Other Commodities   Na          1417        Uganda       NaN   
4      99  Other Commodities   Na          2035         Benin       NaN   

        value  
0  371.280801  
1   18.251164  
2   21.844100  
3    8.034605  
4    1.812666  
  pc_code     pc_description unit  country_code  country_name  quantity  \
0      99  Other Commodities   Na           599   Unspecified       NaN   
1      99  Other Commodities   Na          1213         Kenya       NaN   
2      99  Other Commodities   Na          1395  Tanzania Rep       NaN   
3      99  Other Commodities   Na          1417        Uganda       NaN   
4      99  Other Commodities   Na          2035         Benin       NaN   

  

> Column Descriptions:  
- pc_code: Integer, Principal Commodity Code 
- pc: String, Principal Commodity Name
- unit: String, measurement of quantity
- country_code: Integer, country code
- country_name: String, country name
- quantity: Integer, quantify of export or import
- value: Integer, monetary valeu of the quantity (in million USD)

In [61]:
# to merge all in one
columns = data_ep_2014.columns.append(pd.Index(['year', 'export_import']))

In [62]:
# export_import field, 1 means export, 2 means import
data_ep_2014 = data_ep_2014.assign(year=2014).assign(export_import=1)
data_ip_2014 = data_ip_2014.assign(year=2014).assign(export_import=2)
data_ep_2015 = data_ep_2015.assign(year=2015).assign(export_import=1)
data_ip_2015 = data_ip_2015.assign(year=2015).assign(export_import=2)
data_ep_2016 = data_ep_2016.assign(year=2016).assign(export_import=1)
data_ip_2016 = data_ip_2016.assign(year=2016).assign(export_import=2)

data = pd.DataFrame(columns=columns)
data = reduce(lambda x, y: x.append(y, ignore_index=True), [
    data_ep_2014, data_ep_2015, data_ep_2016,
    data_ip_2014, data_ip_2015, data_ip_2016,
])

print data.sample(n=7)

      pc_code                  pc_description unit  country_code country_name  \
38903      P9  Oth Txtl Yrn, Fbric Mdup Artcl   Na         15352       Serbia   
5011       F8                Finished Leather  Kgs         12239        Macao   
74180      G5  Pearl, Precs, Semiprecs Stones   Na          7171       Guyana   
10888      K7             Packaging Materials  Ton          8091   Costa Rica   
60717      Q8              Rmg Manmade Fibres   Na         13421          U K   
45994      F2          Bulk Minerals And Ores   Na          9103  Dominic Rep   
8342       I7  Residul Chemicl And Alled Prod   Na          6427      Uruguay   

       quantity       value  year  export_import  
38903       NaN    0.109618  2015              1  
5011       44.0    0.006654  2014              1  
74180       NaN    1.084772  2015              2  
10888     322.0    1.133218  2014              1  
60717       NaN  408.071158  2016              1  
45994       NaN    0.114298  2016            

## Exploration

In [84]:
data['unit'] = map(lambda x : None if x == 'Na' else x, data['unit'])

In [85]:
missing_value = map(lambda x : sum(data[x].isnull()), data.columns)

In [90]:
print "there are {} observations and {} features".format(data.shape[0], data.shape[1])
print(pd.DataFrame({'name':data.columns, 'missing':missing_value}))

there are 91440 observations and 9 features
   missing            name
0        0         pc_code
1        0  pc_description
2    48520            unit
3        0    country_code
4        0    country_name
5    48523        quantity
6        0           value
7        0            year
8        0   export_import


In [75]:
print "pc_code value range is " + str(data['pc_code'].unique())

pc_code value range is ['99' 'A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'A7' 'A8' 'A9' 'B1' 'B2' 'B3' 'B4' 'B5'
 'B6' 'B7' 'B8' 'B9' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'C7' 'C8' 'C9' 'D1' 'D2'
 'D3' 'D4' 'D5' 'D6' 'D7' 'D8' 'D9' 'E1' 'E2' 'E3' 'E4' 'E5' 'E6' 'E7' 'E8'
 'E9' 'F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7' 'F8' 'F9' 'G1' 'G2' 'G3' 'G4' 'G5'
 'G6' 'G7' 'G8' 'G9' 'H1' 'H2' 'H3' 'H4' 'H5' 'H6' 'H7' 'H8' 'H9' 'I1' 'I2'
 'I3' 'I4' 'I5' 'I6' 'I7' 'I8' 'I9' 'J1' 'J2' 'J3' 'J4' 'J5' 'J6' 'J7' 'J8'
 'J9' 'K1' 'K2' 'K3' 'K4' 'K5' 'K6' 'K7' 'K8' 'K9' 'L1' 'L2' 'L3' 'L4' 'L5'
 'L6' 'L7' 'L8' 'L9' 'M1' 'M2' 'M3' 'M4' 'M5' 'M6' 'M7' 'M8' 'M9' 'N1' 'N2'
 'N3' 'N4' 'N5' 'N6' 'N7' 'N8' 'N9' 'O1' 'O2' 'O3' 'O4' 'O5' 'O6' 'O7' 'O8'
 'O9' 'P1' 'P2' 'P3' 'P4' 'P5' 'P6' 'P7' 'P8' 'P9' 'Q1' 'Q2' 'Q3' 'Q4' 'Q5'
 'Q6' 'Q7' 'Q8' 'Q9' 'R1' 'R2' 'R3' 'R4' 'R5' 'R6' 'R7' 'R8' 'R9' 'S1' 'S2'
 'S3' 'S4' 'S6' 'S5']


In [91]:
print "unit value range is " + str(data['unit'].unique())

unit value range is [None 'Kgs' 'Ton' 'Ltr' 'Nos' 'Sqm']


In [92]:
print "country_code value range is " + str(data['country_code'].unique())

country_code value range is [  599  1213  1395  1417  2035  2050  2057  2067  2069  2087  2141  2199
  2241  2255  2257  2289  2345  2353  2399  2459  3011  3041  3053  3063
  3085  3102  3115  3116  3117  3143  3149  3167  3169  3227  3229  3243
  3249  3267  3269  3291  3339  3349  3355  3357  3363  3371  3385  3461
  4005  4111  4231  4265  4365  4381  4407  4463  5059  5423  6015  6039
  6043  6073  6083  6109  6317  6319  6427  6433  7012  7013  7020  7023
  7029  7031  7105  7159  7171  7203  7373  7375  7379  7405  8091  8113
  8165  8177  8287  9037  9045  9065  9093  9103  9161  9175  9253  9259
  9277  9311  9331  9383  9439 10001 10017 10027 10038 10047 10056 10077
 10089 10127 10179 10187 10189 10205 10214 10217 10223 10245 10252 10256
 10258 10261 10273 10285 10309 10315 10323 10359 10361 10369 10397 10403
 10437 10447 11025 11191 11207 11219 11225 11301 11335 11351 11391 11419
 11453 12075 12133 12163 12195 12215 12239 12247 12281 12329 12401 12431
 13019 13033 13101 1312

In [110]:
print "country_name value range is " + str(np.sort(data['country_name'].unique()))

country_name value range is ['Afghanistan' 'Albania' 'Algeria' 'Ameri Samoa' 'Andorra' 'Angola'
 'Anguilla' 'Antartica' 'Antigua' 'Argentina' 'Armenia' 'Aruba' 'Australia'
 'Austria' 'Azerbaijan' 'Bahamas' 'Baharain Is' 'Bangladesh Pr' 'Barbados'
 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan' 'Bolivia'
 'Bosnia-Hrzgovin' 'Botswana' 'Br Virgn Is' 'Brazil' 'Brunei' 'Bulgaria'
 'Burkina Faso' 'Burundi' 'C Afri Rep' 'Cambodia' 'Cameroon' 'Canada'
 'Canary Is' 'Cape Verde Is' 'Cayman Is' 'Chad' 'Channel Is' 'Chile'
 'China P Rp' 'Christmas Is.' 'Cocos Is' 'Colombia' 'Comoros'
 'Congo D. Rep.' 'Congo P Rep' 'Cook Is' 'Costa Rica' "Cote D' Ivoire"
 'Croatia' 'Cuba' 'Cyprus' 'Czech Republic' 'Denmark' 'Djibouti'
 'Dominic Rep' 'Dominica' 'East Timor' 'Ecuador' 'Egypt A Rp' 'El Salvador'
 'Equtl Guinea' 'Eritrea' 'Estonia' 'Ethiopia' 'Falkland Is' 'Faroe Is.'
 'Fiji Is' 'Finland' 'Fr Guiana' 'Fr Polynesia' 'Fr S Ant Tr' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Gibraltar

### create a dict of country_name and coutry_code

In [170]:
code_name = data[['country_code', 'country_name']]
print code_name.sample(n=3)

       country_code country_name
63179         15430   Uzbekistan
83967         13193      Ireland
27767         16251        Malta


In [172]:
print data.sample(n=3)

      pc_code                pc_description unit  country_code country_name  \
86528      K7           Packaging Materials  Ton         11219       Kuwait   
11135      K8         Plastic Raw Materials  Ton         15003      Albania   
54324      L9  Tin And Products Made Of Tin  Ton         10205        Japan   

       quantity     value  year  export_import  
86528       2.0  0.007682  2016              2  
11135       1.0  0.005198  2014              1  
54324       0.0  0.006478  2016              1  


In [174]:
print code_name[code_name['country_code'] == 11219]

       country_code country_name
137           11219       Kuwait
288           11219       Kuwait
402           11219       Kuwait
544           11219       Kuwait
690           11219       Kuwait
786           11219       Kuwait
873           11219       Kuwait
960           11219       Kuwait
1154          11219       Kuwait
1305          11219       Kuwait
1419          11219       Kuwait
1553          11219       Kuwait
1621          11219       Kuwait
1680          11219       Kuwait
1773          11219       Kuwait
1867          11219       Kuwait
1964          11219       Kuwait
2057          11219       Kuwait
2161          11219       Kuwait
2356          11219       Kuwait
2488          11219       Kuwait
2607          11219       Kuwait
2712          11219       Kuwait
2817          11219       Kuwait
2966          11219       Kuwait
3136          11219       Kuwait
3251          11219       Kuwait
3351          11219       Kuwait
3495          11219       Kuwait
3613      